In [84]:
import os.path
import random
import time
from functools import partial
from tkinter import *
import sys
from lib.EightPuzzle import EightPuzzle

ModuleNotFoundError: No module named 'lib.EightPuzzle'

In [85]:
root = Tk()

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
puzzle = EightPuzzle(tuple(state))
solution = None

b = [None] * 9

In [86]:
def solve():
    """
    Solves the puzzle using astar_search using puzzle array
    p contains the input puzzle
    goal
    
    return an array of this structure
    ['UP', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'DOWN', 'RIGHT', 'DOWN']
    """
    global puzzle
    p = puzzle.initial
    
    #implement the algorithm here
    
    return ['UP', 'DOWN', 'LEFT', 'RIGHT']

In [90]:
##
from queue import PriorityQueue

class Node:
    def __init__(self, state, parent=None, action=None, cost=0, heuristic=0):
        self.state = state  # Current state of the puzzle
        self.parent = parent  # Parent node
        self.action = action  # Action taken to reach this state from the parent
        self.cost = cost  # Cost to reach this state from the initial state
        self.heuristic = heuristic  # Heuristic value of this state
        self.priority = cost + heuristic  # Priority for A* search

    def __lt__(self, other):
        return self.priority < other.priority

def heuristic(state, goal):
    """
    Heuristic function to estimate the cost from the current state to the goal state.
    In this case, we can use the Manhattan distance heuristic.
    """
    return sum(abs(state[i] // 3 - goal[i] // 3) + abs(state[i] % 3 - goal[i] % 3) for i in range(9))

def solve():
    global puzzle
    initial_state = puzzle.initial
    goal_state = puzzle.goal

    frontier = PriorityQueue()
    frontier.put(Node(initial_state, None, None, 0, heuristic(initial_state, goal_state)))

    explored = set()

    while not frontier.empty():
        current_node = frontier.get()
        current_state = current_node.state

        if current_state == goal_state:
            # Reconstruct and return the path
            path = []
            while current_node.action:
                path.append(current_node.action)
                current_node = current_node.parent
            return path[::-1]

        explored.add(current_state)

        for action in ['UP', 'DOWN', 'LEFT', 'RIGHT']:  # Generate successor states
            child_state = puzzle.result(current_state, action)
            if child_state not in explored:
                cost = current_node.cost + 1  # Uniform cost for all actions
                heuristic_val = heuristic(child_state, goal_state)
                child_node = Node(child_state, current_node, action, cost, heuristic_val)
                frontier.put(child_node)

    return None


In [88]:
from queue import PriorityQueue

class Node:
    def __init__(self, state, parent=None, action=None, cost=0, heuristic=0):
        self.state = state  # Current state of the puzzle
        self.parent = parent  # Parent node
        self.action = action  # Action taken to reach this state from the parent
        self.cost = cost  # Cost to reach this state from the initial state
        self.heuristic = heuristic  # Heuristic value of this state
        self.priority = cost + heuristic  # Priority for A* search

    def __lt__(self, other):
        return self.priority < other.priority

def heuristic(state, goal):
    """
    Heuristic function to estimate the cost from the current state to the goal state.
    In this case, we can use the Manhattan distance heuristic.
    """
    return sum(abs(state[i] // 3 - goal[i] // 3) + abs(state[i] % 3 - goal[i] % 3) for i in range(9))

def solve(puzzle):
    initial_state = puzzle.initial
    goal_state = puzzle.goal

    frontier = PriorityQueue()
    frontier.put(Node(initial_state, None, None, 0, heuristic(initial_state, goal_state)))

    explored = set()

    while not frontier.empty():
        current_node = frontier.get()
        current_state = current_node.state

        if current_state == goal_state:
            # Reconstruct and return the path
            path = []
            while current_node.action:
                path.append(current_node.action)
                current_node = current_node.parent
            return path[::-1]

        explored.add(tuple(current_state))  # Convert state to tuple for hashability

        for action in ['UP', 'DOWN', 'LEFT', 'RIGHT']:  # Generate successor states
            child_state = puzzle.result(list(current_state), action)  # Convert back to list
            if child_state is not None and tuple(child_state) not in explored:
                cost = current_node.cost + 1  # Uniform cost for all actions
                heuristic_val = heuristic(child_state, goal_state)
                child_node = Node(child_state, current_node, action, cost, heuristic_val)
                frontier.put(child_node)

    return None


# Example of a puzzle class
class Puzzle:
    def __init__(self, initial, goal):
        self.initial = initial
        self.goal = goal

    def result(self, state, action):
        # Implement the result function to get the new state after applying the action
        pass

# Define your puzzle and its initial and goal states
initial_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
puzzle = Puzzle(initial_state, goal_state)

# Call the solve function to get the solution
solution = solve(puzzle)

# Print the solution
print(solution)


None


In [89]:

def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()


def solve_steps():
    """Solves the puzzle step by step"""

    global puzzle
    global solution
    global state
    solution = solve()
    print(solution)

    for move in solution:
        state = puzzle.result(state, move)
        create_buttons()
        root.update()
        root.after(1, time.sleep(0.75))


def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'),
                            command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))


def create_buttons():
    """Creates dynamic buttons"""

    # TODO: Find a way to use grid_forget() with a for loop for initialization
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 0))
    b[0].grid(row=0, column=0, ipady=40)
    b[1] = Button(root, text=f'{state[1]}' if state[1] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 1))
    b[1].grid(row=0, column=1, ipady=40)
    b[2] = Button(root, text=f'{state[2]}' if state[2] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 2))
    b[2].grid(row=0, column=2, ipady=40)
    b[3] = Button(root, text=f'{state[3]}' if state[3] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 3))
    b[3].grid(row=1, column=0, ipady=40)
    b[4] = Button(root, text=f'{state[4]}' if state[4] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 4))
    b[4].grid(row=1, column=1, ipady=40)
    b[5] = Button(root, text=f'{state[5]}' if state[5] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 5))
    b[5].grid(row=1, column=2, ipady=40)
    b[6] = Button(root, text=f'{state[6]}' if state[6] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 6))
    b[6].grid(row=2, column=0, ipady=40)
    b[7] = Button(root, text=f'{state[7]}' if state[7] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 7))
    b[7].grid(row=2, column=1, ipady=40)
    b[8] = Button(root, text=f'{state[8]}' if state[8] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 8))
    b[8].grid(row=2, column=2, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(init))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)


def init():
    """Calls necessary functions"""

    global state
    global solution
    state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    scramble()
    create_buttons()
    create_static_buttons()


init()
root.mainloop()


AttributeError: 'Puzzle' object has no attribute 'actions'

In [71]:
from tkinter import *
from functools import partial
import random
import time
from search import *

root = Tk()
root.title("8 Puzzle")

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
b = [None] * 9
puzzle = EightPuzzle(tuple(state))
solution = []

def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()


def solve_steps():
    """Solves the puzzle step by step"""

    global puzzle
    global solution
    global state
    solution = solve()
    print(solution)

    for move in solution:
        state = puzzle.result(state, move)
        create_buttons()
        root.update()
        root.after(1, time.sleep(0.75))


def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'),
                            command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))


def create_buttons():
    """Creates dynamic buttons"""

    for i in range(9):
        b[i] = Button(root, text=f'{state[i]}' if state[i] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                      command=partial(exchange, i))
        b[i].grid(row=i // 3, column=i % 3, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(scramble))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)


def init():
    """Calls necessary functions"""

    global state
    global solution
    state = [int(input(f"Enter tile at position {i + 1}: ")) for i in range(9)]
    scramble()
    create_buttons()
    create_static_buttons()


init()
root.mainloop()


ModuleNotFoundError: No module named 'search'

In [73]:
from tkinter import *
from functools import partial
import random
import time

class EightPuzzle:
    def __init__(self, initial):
        self.initial = initial
        self.goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only four possible actions
        in any given state of the environment """
        possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        index_blank_square = state.index(0)

        if index_blank_square % 3 == 0:
            possible_actions.remove('LEFT')
        if index_blank_square < 3:
            possible_actions.remove('UP')
        if index_blank_square % 3 == 2:
            possible_actions.remove('RIGHT')
        if index_blank_square > 5:
            possible_actions.remove('DOWN')

        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """

        # blank is the index of the blank square
        blank = state.index(0)
        new_state = list(state)

        delta = {'UP': -3, 'DOWN': 3, 'LEFT': -1, 'RIGHT': 1}
        neighbor = blank + delta[action]
        new_state[blank], new_state[neighbor] = new_state[neighbor], new_state[blank]

        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        return state == self.goal

def bfs(problem):
    """Breadth-First Search"""
    frontier = [(problem.initial, [])]
    explored = set()

    while frontier:
        state, actions = frontier.pop(0)
        if problem.goal_test(state):
            return actions
        explored.add(state)
        for action in problem.actions(state):
            child_state = problem.result(state, action)
            if child_state not in explored:
                frontier.append((child_state, actions + [action]))

    return None

root = Tk()
root.title("8 Puzzle")

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
b = [None] * 9
puzzle = EightPuzzle(tuple(state))
solution = []

def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()


def solve_steps():
    """Solves the puzzle step by step"""

    global puzzle
    global solution
    global state
    solution = bfs(puzzle)
    print(solution)

    for move in solution:
        state = puzzle.result(state, move)
        create_buttons()
        root.update()
        root.after(1, time.sleep(0.75)))


def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'),
                            command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))


def create_buttons():
    """Creates dynamic buttons"""

    for i in range(9):
        b[i] = Button(root, text=f'{state[i]}' if state[i] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                      command=partial(exchange, i))
        b[i].grid(row=i // 3, column=i % 3, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(scramble))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)


def init():
    """Calls necessary functions"""

    global state
    global solution
    state = [int(input(f"Enter tile at position {i + 1}: ")) for i in range(9)]
    scramble()
    create_buttons()
    create_static_buttons()


init()
root.mainloop()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 146)

In [74]:
###Implemented solutu
class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        return 0  # Placeholder value, replace with your implementation


class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board, where one of the
    squares is a blank. A state is represented as a tuple of length 9, where  element at
    index i represents the tile number  at index i (0 if it's an empty square) """

    def __init__(self, initial):
        """ Define goal state and initialize a problem """
        goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)
        super().__init__(initial, goal_state)

    def find_blank_square(self, state):
        """Return the index of the blank square in a given state"""

        return state.index(0)

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only four possible actions
        in any given state of the environment """

        possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        index_blank_square = self.find_blank_square(state)

        if index_blank_square % 3 == 0:
            possible_actions.remove('LEFT')
        if index_blank_square < 3:
            possible_actions.remove('UP')
        if index_blank_square % 3 == 2:
            possible_actions.remove('RIGHT')
        if index_blank_square > 5:
            possible_actions.remove('DOWN')

        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """

        # blank is the index of the blank square
        blank = self.find_blank_square(state)
        new_state = list(state)

        delta = {'UP': -3, 'DOWN': 3, 'LEFT': -1, 'RIGHT': 1}
        neighbor = blank + delta[action]
        new_state[blank], new_state[neighbor] = new_state[neighbor], new_state[blank]

        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """

        return state == self.goal

    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j] != 0:
                    inversion += 1

        return inversion % 2 == 0

    def h(self, node):
        """ Return the heuristic value for a given state. Default heuristic function used is 
        h(n) = number of misplaced tiles """

        return sum(s != g for (s, g) in zip(node.state, self.goal))

def astar_search(problem, h):
    """A* search"""
    frontier = [(h(problem.initial, problem.goal), Node(problem.initial))]
    explored = set()

    while frontier:
        frontier.sort(key=lambda x: x[0])
        f, node = frontier.pop(0)
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)
            if child_state not in explored:
                child = Node(child_state, node, action, node.path_cost + 1)
                frontier.append((h(child_state, problem.goal) + child.path_cost, child))

    return None

def misplaced_tiles(state, goal):
    """Heuristic function: number of misplaced tiles"""
    return sum(s != g for (s, g) in zip(state, goal))

# Example usage:
initial_state = (1, 2, 3, 4, 0, 5, 6, 7, 8)
eight_puzzle = EightPuzzle(initial_state)
if eight_puzzle.check_solvability(initial_state):
    solution_node = astar_search(eight_puzzle, misplaced_tiles)
    if solution_node:
        actions = []
        while solution_node.parent:
            actions.append(solution_node.action)
            solution_node = solution_node.parent
        actions.reverse()
        print("Solution found with", len(actions), "steps:", actions)
    else:
        print("No solution found.")
else:
    print("The initial state is unsolvable.")
    print("Unreslved issue:")


AttributeError: 'Node' object has no attribute 'path_cost'